In [5]:
import openai
import outlines
client = openai.OpenAI(
    base_url="https://api.deepseek.com",
    api_key="sk-8c887e6454f741cc9553da2be62487af",
)
model = outlines.from_openai(client, "deepseek-reasoner")

In [3]:

from db_tools import DatabaseTools
from fetch_kline_daily import get_market_snapshot
db_manager = DatabaseTools()

2025-12-10 20:11:20,916 | 20368 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=31759416
2025-12-10 20:11:20,983 | 20368 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2025-12-10 20:11:21,027 - db_schema - INFO - 成功初始化数据库: investment_portfolio.db


In [ ]:
market_state = db_manager.get_market_place()

In [16]:
f"portfolio: {portfolio['positions'][0]['value']:.2f}"


'portfolio: 1.91'

In [6]:
portfolios = db_manager.get_portfolios_by_account('user_001') if db_manager else []
# 构建包含持仓信息的提示
account_info = dict()
account_info['initial_capital'] = portfolios[0]['initial_capital']
account_info['total_return'] = portfolios[0]['total_return'] / account_info['initial_capital']
positions = db_manager.get_positions_by_portfolio(portfolios[0]['portfolio_id'])
portfolio = dict()
portfolio['total_value'] = portfolios[0]['total_value']
portfolio['cash'] = portfolios[0]['cash']
portfolio['positions'] = []
for item in positions:
    portfolio['positions'].append({
        'name': item['name'],
        'code': item['code'],
        'quantity': item['quantity'],
        'value': get_market_snapshot(item['code']),
    })

2025-12-10 20:13:14,907 - db_schema - INFO - 查询到账户user_001的1个投资组合记录
2025-12-10 20:13:14,913 - db_schema - INFO - 查询到投资组合port_user_001_2025-12-09_1765246369的4个持仓记录


2025-12-10 20:13:14,921 | 20368 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=18, host=127.0.0.1, port=11111, user_id=31759416
2025-12-10 20:13:15,010 | 20368 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=18
2025-12-10 20:13:15,014 | 20368 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=19, host=127.0.0.1, port=11111, user_id=31759416
2025-12-10 20:13:15,040 | 20368 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=19
2025-12-10 20:13:15,044 | 20368 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=20, host=127.0.0.1, port=11111, user_id=31759416
2025-12-10 20:13:15,123 | 20368 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=20
2025-12-10 20:13:15,126 | 20368 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=21, host=127.0.0.1, port=11111, user_id=31759416
2025-12-10 20:13:15,175 | 20368 | [open_context_base.py] on_d

In [17]:
from outlines import Template
from pathlib import Path
template = Template.from_file(Path("trading_prompt.jinja"))

In [18]:
prompt = template(market_state=market_state, account_info=account_info, portfolio=portfolio)

In [19]:
print(prompt)


你是一名专业的ETF交易员。请基于提供的市场数据、账户状态、当前持仓和交易策略，通过技术面分析，为每个ETF生成具体的交易决策。

**输入数据：**
- **市场数据：** 包含每个ETF的当前价格、24小时涨跌幅，以及技术指标（SMA5, SMA10, SMA20 RSI）。
- **账户状态：** 包含初始资金、总价值、现金和总收益率。
- **当前持仓：** 列出所有当前持有的ETF及其数量、平均成本。
- **交易策略：** 定义了信号类型、风险管理规则、仓位大小分级、买入和退出策略。

**你的任务：**
1.  **分析：** 针对每个ETF（包括已持仓和未持仓的），结合其技术指标（价格、SMA、RSI）和市场状态，评估其是否符合“波段中的低点”（买入信号）或“波段中的高点”（卖出/清仓信号）。
2.  **决策：** 根据分析，为每个ETF确定一个交易信号（`buy_to_enter`， `sell`， `hold`， `close_position`）。
3.  **规划：** 如果信号是`buy_to_enter`或`sell`，需计算具体的交易数量、设置盈利目标和止损点，并评估信心水平。
    -   **交易数量计算：** 基于“每次交易风险”（1-5%）、账户总价值和ETF当前价格，计算风险金额，然后转换为股数。根据市场情况选择保守、中等或激进的风险等级。
    -   **盈利目标与止损：** 基于技术分析（如支撑/阻力位、波动率）设置具体的`profit_target`和`stop_loss`价格。
    -   **信心水平：** 根据信号强度和技术指标的吻合度，给出一个0到1之间的`confidence`值。
4.  **风险管理：** 确保决策符合策略：总持仓不超过4个；每次交易风险控制在1-5%；快速关闭亏损持仓，让利润奔跑。

**输出要求：**
- **格式：** 严格仅输出一个JSON对象。
- **内容：** JSON对象的键为ETF代码（例如`SH.512150`）。每个ETF代码对应的值是一个包含以下字段的对象：
    -   `signal`: 交易信号，必须是`"buy_to_enter"`、`"sell"`、`"hold"`、`"close_position"`中的一个。
    -   `quant

In [20]:
print(model(prompt))

2025-12-10 20:20:26,134 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


{
  "SH.512150": {
    "signal": "hold",
    "quantity": 0,
    "leverage": 10,
    "profit_target": 0,
    "stop_loss": 0,
    "confidence": 0.6,
    "justification": "价格高于短期均线，SMA5>SMA10≈SMA20，RSI中性偏多，趋势初步转多，持有。"
  },
  "SH.516390": {
    "signal": "hold",
    "quantity": 0,
    "leverage": 10,
    "profit_target": 0,
    "stop_loss": 0,
    "confidence": 0.5,
    "justification": "价格略高于短期均线但低于SMA20，RSI中性，趋势震荡，暂无明确卖出信号，持有。"
  },
  "SH.515210": {
    "signal": "hold",
    "quantity": 0,
    "leverage": 10,
    "profit_target": 0,
    "stop_loss": 0,
    "confidence": 0.4,
    "justification": "价格低于所有均线，均线空头排列，RSI 41.0显示弱势，未出现企稳信号，不符合买入条件。"
  },
  "SH.515790": {
    "signal": "hold",
    "quantity": 0,
    "leverage": 10,
    "profit_target": 0,
    "stop_loss": 0,
    "confidence": 0.5,
    "justification": "价格在短期均线附近，均线交织，RSI中性，方向不明，观望。"
  },
  "SH.512760": {
    "signal": "hold",
    "quantity": 0,
    "leverage": 10,
    "profit_target": 0,
    "stop_loss": 0,
    "confidence": 0.6